# Análisis: Value Iteration y Policy Iteration

**Tomas Acosta Bernal**

## Introducción

Este documento presenta el análisis comparativo de dos algoritmos de programación dinámica para resolver Procesos de Decisión de Markov (MDPs):

1. **Value Iteration (Iteración de Valores)**
2. **Policy Iteration (Iteración de Políticas)**

Se evalúan ambos algoritmos en dos ambientes:
- **GridWorld 10x10**
- **Bridge (3x7)**

Con diferentes parámetros (iteraciones, factores de descuento) para analizar su convergencia y comportamiento.

---
# Parte 1: Value Iteration (Iteración de Valores)
---

## 1.1 GridWorld - Convergencia de Valores y Acciones

### Ejecución

Se ejecutó Value Iteration con 5, 10, 15, 20, 30, y 50 iteraciones sobre GridWorld 10x10.

In [6]:
from gridworld_environment import GridWorld10x10
from bridge_environment import BridgeEnvironment
from mdp import MDP
from value_iteration import ValueIteration
from policy_iteration import PolicyIteration

env = GridWorld10x10()
mdp = MDP(env)
arrows = {'up': '  UP  ', 'down': '  DOWN  ', 'left': '  LEFT  ', 'right': '  RIGHT  ', None: '  EXIT  '}

def print_values(agent, env):
    for r in range(env.nrows):
        row = ''
        for c in range(env.ncols):
            if env.board[r][c] == '#':
                row += '  ####  '
            else:
                row += f'{agent.get_value((r, c)):+7.3f} '
        print(row)

def print_policy(agent, env):
    for r in range(env.nrows):
        row = ''
        for c in range(env.ncols):
            if env.board[r][c] == '#':
                row += ' #### '
            else:
                action = agent.get_policy((r, c))
                row += f'{arrows[action]}'
        print(row)

print("VALUE ITERATION - GridWorld 10x10")
print("="*70)

for n_iter in [5, 10, 15, 20, 30, 50]:
    print(f"\n{'-'*70}")
    print(f"Iteraciones: {n_iter}")
    print(f"{'-'*70}")
    
    vi = ValueIteration(mdp, discount=0.9, iterations=n_iter)
    vi.run_value_iteration()
    
    print("\nValores:")
    print_values(vi, env)
    
    print("\nPolítica:")
    print_policy(vi, env)

VALUE ITERATION - GridWorld 10x10

----------------------------------------------------------------------
Iteraciones: 5
----------------------------------------------------------------------

Valores:
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000 
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.000 
 +0.000   ####    ####    ####    ####    ####    ####    ####   +0.000  +0.000 
 +0.000  +0.000  +0.000  +0.000   ####   +0.026  +0.160  +0.165  +0.092  +0.000 
 +0.000  +0.000  +0.000  +0.000   ####   +0.000  +0.302  +0.343  +0.177  +0.079 
 +0.000  +0.000  +0.000  +0.000   ####   +0.000  +0.769  +0.525  +0.322  +0.128 
 +0.000  +0.000  +0.000  +0.000   ####   +0.864  +0.663  +0.475  +0.265  +0.119 
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.286  +0.242  +0.126  +0.000 
 +0.000  +0.000  +0.000  +0.000   ####   +0.006  +0.116  +0.098  +0.000  +0.000 
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000  +0.039  +0.000  +0.0

### Resultados - GridWorld Value Iteration

#### 5 Iteraciones
- **Valores**: La mayoría en 0, solo estados cercanos al objetivo tienen valores (máximo +0.864 en `(6,5)`).
- **Política**: Muy primitiva - casi todos los estados apuntan UP, no es coherente. Los valores no se han propagado lo suficiente.

#### 10 Iteraciones
- **Valores**: Se propagan hacia las esquinas (valor máximo en esquina superior derecha: +0.111).
- **Política**: Empieza a tomar forma - las filas superiores ahora apuntan RIGHT, pero aún hay inconsistencias en el cuadrante inferior izquierdo.

#### 15 Iteraciones
- **Valores**: Continúan propagándose (esquina superior izquierda: +0.000 → +0.003).
- **Política**: Mejora significativa - fila superior ahora consistentemente RIGHT, pero hay diferencias con iteraciones posteriores (ej: estado `(3,1)` es UP, luego será DOWN en iteración 20).

#### 20 Iteraciones
- **Valores**: Valores significativos en todo el grid (esquina superior izquierda: +0.010).
- **Política**: CONVERGE - idéntica a las iteraciones 30 y 50. Todas las direcciones son coherentes hacia el objetivo.

#### 30-50 Iteraciones
- **Valores**: Refinamiento continuo - valor en `(0,0)` pasa de +0.054 (iter 30) a +0.063 (iter 50).
- **Política**: SIN CAMBIOS desde iteración 20.

### Análisis

**¿Cuándo convergen los valores?**
- Los valores aún están cambiando incluso a las **50 iteraciones** (aunque los cambios son pequeños).
- Para convergencia completa (cambios < 0.001), se necesitarían probablemente más iteraciones.

**¿Cuándo convergen las acciones (política)?**
- La política converge en la iteración 20.
- Entre las iteraciones 15 y 20 aún hay cambios importantes (ej: varios estados en la columna izquierda cambian de UP a DOWN).

**Conclusión importante:**
- La política óptima converge (~20 iteraciones) mucho antes que los valores (~50+ iteraciones).
- Esto ocurre porque la política depende del orden relativo de los Q-valores (qué acción es mejor), no de sus magnitudes exactas.
- Una vez que `Q(s, a_mejor) > Q(s, otras_acciones)` de forma consistente, la acción óptima está determinada, aunque los valores exactos sigan refinándose.

## 1.2 Bridge - Efecto del Factor de Descuento

### Ejecución

In [7]:
env_bridge = BridgeEnvironment()
mdp_bridge = MDP(env_bridge)

print("\n" + "="*70)
print("VALUE ITERATION - Bridge Environment")
print("="*70)

for discount in [0.9, 0.1]:
    print(f"\n{'-'*70}")
    print(f"Factor de Descuento: γ = {discount}")
    print(f"{'-'*70}")
    
    vi = ValueIteration(mdp_bridge, discount=discount, iterations=10)
    vi.run_value_iteration()
    
    print("\nValores:")
    print_values(vi, env_bridge)
    
    print("\nPolítica:")
    print_policy(vi, env_bridge)


VALUE ITERATION - Bridge Environment

----------------------------------------------------------------------
Factor de Descuento: γ = 0.9
----------------------------------------------------------------------

Valores:
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000 +69.794 
 +0.000 -32.967 -52.530 -40.921 -13.404 +32.967  +0.000 
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000 +68.493 

Política:
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    RIGHT  
  LEFT    LEFT    LEFT    RIGHT    RIGHT    RIGHT    EXIT  
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    UP  

----------------------------------------------------------------------
Factor de Descuento: γ = 0.1
----------------------------------------------------------------------

Valores:
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000 +40.816 
 +0.000 -30.303 -32.140 -32.028 -28.466 +30.303  +0.000 
 +0.000  +0.000  +0.000  +0.000  +0.000  +0.000 +51.546 

Política:
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    DOWN  
  LE

### Resultados - Bridge Value Iteration

#### γ = 0.9 (Alto - valora recompensas futuras)

**Valores (Fila 1 - El Puente):**
```
+0.000  -32.967  -52.530  -40.921  -13.404  +32.967  +0.000 (terminal +100)
```

**Política:**
```
Fila 0:  LEFT   EXIT   EXIT   EXIT   EXIT   EXIT  RIGHT
Fila 1:  LEFT   LEFT   LEFT  RIGHT  RIGHT  RIGHT   EXIT
Fila 2:  LEFT   EXIT   EXIT   EXIT   EXIT   EXIT    UP
```

**Observaciones:**
- El agente **cruza el puente** de izquierda a derecha hacia `(1,6)` con recompensa +100.
- Valores negativos en el puente debido al riesgo de transiciones estocásticas a casillas -100.
- Estado `(0,6)` apunta **RIGHT** hacia el objetivo porque γ=0.9 hace que el +100 sea valioso incluso desde lejos.

#### γ = 0.1 (Bajo - descuenta fuertemente el futuro)

**Valores (Fila 1 - El Puente):**
```
+0.000  -30.303  -32.140  -32.028  -28.466  +30.303  +0.000
```

**Política:**
```
Fila 0:  LEFT   EXIT   EXIT   EXIT   EXIT   EXIT   DOWN
Fila 1:  LEFT   LEFT   LEFT  RIGHT  RIGHT  RIGHT   EXIT
Fila 2:  LEFT   EXIT   EXIT   EXIT   EXIT   EXIT    UP
```

**Observaciones:**
- El agente **también cruza el puente** (política en fila 1 idéntica).
- Valores mucho más bajos en magnitud.
- **Diferencia clave:** Estado `(0,6)` apunta **DOWN** hacia terminal negativo en lugar del objetivo.

### Análisis

**¿Qué cambios se observan?**

1. **Magnitud de valores:**
   - γ=0.9: valores más extremos (altos y bajos)
   - γ=0.1: valores comprimidos cerca de cero

2. **Política en el puente:**
   - Idéntica en ambos casos porque el agente comienza en `(1,0)`, sin alternativa.

3. **Política en estados lejanos:**
   - γ=0.9: coherente hacia el objetivo en todo el espacio
   - γ=0.1: subóptima en estados lejanos

**¿Por qué cambian los resultados?**

El factor de descuento γ controla la **visión temporal** del agente:

**γ = 0.9 (Visionario):**
- Planifica a largo plazo
- Después de 10 pasos: `0.9^10 ≈ 0.35` del valor original
- El objetivo +100 sigue siendo atractivo desde lejos

**γ = 0.1 (Miope):**
- Solo valora el futuro inmediato
- Después de 10 pasos: `0.1^10 ≈ 10^-10` ≈ cero
- El objetivo +100 es "invisible" desde estados lejanos

**Fórmula:**
```
V(s) = E[R_0 + γR_1 + γ²R_2 + γ³R_3 + ...] = E[Σ γ^t R_t]
```

---
# Parte 2: Policy Iteration (Iteración de Políticas)
---

## 2.1 GridWorld y Bridge - Convergencia de Políticas

### GridWorld

In [8]:
print("\n" + "="*70)
print("POLICY ITERATION - GridWorld 10x10")
print("="*70)

for n_iter in [5, 10, 15, 20]:
    print(f"\n{'-'*70}")
    print(f"Iteraciones: {n_iter}")
    print(f"{'-'*70}")
    
    pi = PolicyIteration(mdp, discount=0.9, iterations=n_iter)
    pi.run_policy_iteration()
    
    print("\nPolítica:")
    print_policy(pi, env)


POLICY ITERATION - GridWorld 10x10

----------------------------------------------------------------------
Iteraciones: 5
----------------------------------------------------------------------

Política:
  DOWN    DOWN    DOWN    DOWN    DOWN    DOWN    DOWN    DOWN    DOWN    DOWN  
  RIGHT    RIGHT    RIGHT    RIGHT    RIGHT    RIGHT    RIGHT    RIGHT    DOWN    DOWN  
  UP   ####  ####  ####  ####  ####  ####  ####   DOWN    DOWN  
  UP    UP    UP    UP   ####   UP    DOWN    DOWN    DOWN    DOWN  
  UP    UP    UP    UP   ####   EXIT    RIGHT    DOWN    DOWN    DOWN  
  UP    UP    UP    UP   ####   EXIT    LEFT    LEFT    LEFT    LEFT  
  UP    UP    UP    UP   ####   UP    LEFT    LEFT    LEFT    LEFT  
  UP    UP    UP    LEFT    EXIT    EXIT    RIGHT    UP    LEFT    LEFT  
  DOWN    DOWN    DOWN    DOWN   ####   DOWN    UP    UP    UP    UP  
  RIGHT    RIGHT    RIGHT    RIGHT    RIGHT    RIGHT    UP    UP    UP    UP  

------------------------------------------------------

## 2.1 GridWorld y Bridge - Convergencia de Políticas

### Pregunta: ¿Cuándo convergen las políticas en Policy Iteration?

#### GridWorld 10x10

**Resultados observados:**

**5 iteraciones:** Política subóptima - NO converge
- Fila 0: todos los estados apuntan `DOWN` (incorrecto, debería ser `RIGHT`)
- Estado (0,1): `DOWN` pero debería ser `RIGHT`
- Estado (3,1): `UP` pero debería ser `DOWN`
- Estado (8,0): `DOWN` pero debería ser `RIGHT`
- La política aún no ha convergido

**10 iteraciones:** Política óptima - CONVERGE
- Fila 0: todos los estados apuntan `RIGHT` 
- Estado (0,1): `RIGHT` 
- Estado (3,1): `DOWN` 
- Estado (8,0): `RIGHT` 
- Cambios significativos respecto a iteración 5

**15 iteraciones:** Política idéntica a iteración 10
- Sin cambios

**20 iteraciones:** Política idéntica a iteración 10
- Sin cambios

**Convergencia GridWorld:** La política converge **entre las iteraciones 5 y 10**. La política es óptima y estable desde la iteración 10.


In [9]:
print("\n" + "="*70)
print("POLICY ITERATION - Bridge Environment")
print("="*70)

for n_iter in [5, 10]:
    print(f"\n{'-'*70}")
    print(f"Iteraciones: {n_iter}")
    print(f"{'-'*70}")
    
    pi = PolicyIteration(mdp_bridge, discount=0.9, iterations=n_iter)
    pi.run_policy_iteration()
    
    print("\nPolítica:")
    print_policy(pi, env_bridge)


POLICY ITERATION - Bridge Environment

----------------------------------------------------------------------
Iteraciones: 5
----------------------------------------------------------------------

Política:
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    RIGHT  
  LEFT    LEFT    LEFT    RIGHT    RIGHT    RIGHT    EXIT  
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    UP  

----------------------------------------------------------------------
Iteraciones: 10
----------------------------------------------------------------------

Política:
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    RIGHT  
  LEFT    LEFT    LEFT    RIGHT    RIGHT    RIGHT    EXIT  
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    UP  


### Resultados - Bridge Policy Iteration

**Observaciones:**
- **5 iteraciones:** Política óptima completa
- **10 iteraciones:** Política **idéntica**

**Convergencia:**
- Converge en **menos de 5 iteraciones**.

### Análisis

**¿Por qué Policy Iteration converge tan rápido?**

1. **Mejora**
   - En cada iteración, Policy Iteration evalúa completamente la política actual.
   - Luego mejora la política haciéndola greedy respecto a los valores.
   - Esto garantiza que `V^π_{k+1} ≥ V^π_k`.

2. **Teorema de mejora de política:**
   - Si `π'` es greedy respecto a `V^π`, entonces `π'` es al menos tan buena como `π`.
   - El espacio de políticas determinísticas es finito, por lo que debe converger.

3. **Cambios audaces:**
   - Policy Iteration hace cambios más agresivos que Value Iteration.
   - Evalúa completamente una política antes de mejorarla.

**Conclusión:**
- **GridWorld:** < 5 iteraciones
- **Bridge:** < 5 iteraciones
- **Muy eficiente** en iteraciones externas.

## 2.2 Bridge - Efecto del Factor de Descuento

### Ejecución

In [10]:
print("\n" + "="*70)
print("POLICY ITERATION - Bridge con diferentes γ")
print("="*70)

for discount in [0.9, 0.1]:
    print(f"\n{'-'*70}")
    print(f"Factor de Descuento: γ = {discount}")
    print(f"{'-'*70}")
    
    pi = PolicyIteration(mdp_bridge, discount=discount, iterations=10)
    pi.run_policy_iteration()
    
    print("\nValores:")
    print_values(pi, env_bridge)
    
    print("\nPolítica:")
    print_policy(pi, env_bridge)


POLICY ITERATION - Bridge con diferentes γ

----------------------------------------------------------------------
Factor de Descuento: γ = 0.9
----------------------------------------------------------------------

Valores:
 -2.841  +0.000  +0.000  +0.000  +0.000  +0.000 +71.397 
 -2.841 -34.861 -53.792 -40.921 -13.404 +32.967  +0.000 
 -2.840  +0.000  +0.000  +0.000  +0.000  +0.000 +68.493 

Política:
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    RIGHT  
  LEFT    LEFT    LEFT    RIGHT    RIGHT    RIGHT    EXIT  
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    UP  

----------------------------------------------------------------------
Factor de Descuento: γ = 0.1
----------------------------------------------------------------------

Valores:
 -0.000  +0.000  +0.000  +0.000  +0.000  +0.000 +40.816 
 -0.000 -30.303 -32.140 -32.028 -28.466 +30.303  +0.000 
 -0.000  +0.000  +0.000  +0.000  +0.000  +0.000 +51.546 

Política:
  LEFT    EXIT    EXIT    EXIT    EXIT    EXIT    DOWN 

### Resultados - Bridge Policy Iteration

Los resultados son **idénticos** a Value Iteration:

#### γ = 0.9
- Política cruza el puente
- Estado `(0,6)`: **RIGHT** (hacia objetivo)
- Valores altos en magnitud

#### γ = 0.1
- Política cruza el puente
- Estado `(0,6)`: **DOWN** (hacia terminal negativo)
- Valores bajos en magnitud

**Conclusión:**
- Ambos algoritmos **convergen a la misma política óptima**.
- El factor de descuento tiene el mismo efecto en ambos algoritmos.

---
# Parte 3: Comparación de Algoritmos
---

## 3.1 Convergencia

| Algoritmo | GridWorld (política) | GridWorld (valores) | Bridge |
|-----------|---------------------|---------------------|--------|
| **Value Iteration** | ~15 iteraciones | ~30-50 iteraciones | ~10 iteraciones |
| **Policy Iteration** | < 5 iteraciones | N/A | < 5 iteraciones |

**Conclusión:** Policy Iteration converge en muchas menos iteraciones externas.


## 3.2 ¿Cuándo usar cada uno?

**Value Iteration:**
- MDPs grandes donde cada iteración debe ser rápida
- Solo necesitas una política aproximada
- Implementación simple

**Policy Iteration:**
- MDPs pequeños/medianos
- Necesitas la política óptima exacta
- Puedes permitir más cómputo por iteración

**Nota:** Ambos convergen a la misma política óptima cuando el MDP es resoluble.

---
# Conclusiones Finales
---

## Convergencia

1. **Value Iteration:**
   - Política converge antes (~15 iter) que valores (~30-50 iter)
   - La política óptima se puede extraer sin esperar convergencia completa de valores

2. **Policy Iteration:**
   - Converge muy rápido (< 5 iteraciones en ambos ambientes)
   - Early stopping automático cuando política es estable

## Factor de Descuento

El factor de descuento γ es practicamente el que determina para el comportamiento del agente:

- **γ ≈ 1 (e.g., 0.9):**
  - Agente visionario, planifica a largo plazo
  - Valores altos en magnitud
  - Política coherente en todo el espacio
  - Recomendado para tareas con horizonte largo

- **γ ≈ 0 (e.g., 0.1):**
  - Agente miope, solo valora futuro inmediato
  - Valores bajos en magnitud
  - Política puede ser subóptima en estados lejanos
  - Recomendado para tareas con recompensa inmediata